### Higgs Fit

RooFit example on fitting the 2-photon invariant mass to determine the number of Higgs signal events.

We read the data set consisting of ~ 3000 2-photon invariant mass values from a tetx file (Hgg.txt) which should be in the current directory. 
Otherwise it can be downloaded from here: https://twiki.cern.ch/twiki/pub/Main/ROOTRioTutorial/Hgg.txt

In [ ]:
   TTree tree("tree","tree");
   int nevt = tree.ReadFile("Hgg.txt","x");
   if (nevt <= 0) {
      Error("fitHgg","Error reading data from input file ");
   }
   std::cout << "Read " << nevt << " from the file " << std::endl;

We make now the RooFit model using the RooWorkspace Factory 

In [ ]:
 RooWorkspace w("w");
   w.factory("x[110,160]");  // invariant mass
   
   w.factory("nbackground[10000, 0, 10000]");
   //w.factory("Exponential::z1(x, a1[-1,-10,0])");
   w.var("nbackground")->setVal(nevt);
   w.var("nbackground")->setMin(0.1*nevt);
   w.var("nbackground")->setMax(10*nevt);

   // create exponential model as two components
   w.factory("a1[ 7.5, -500, 500]");
   w.factory("a2[-1.5, -500, 500]");
   w.factory("expr::z('-(a1*x/100 + a2*(x/100)^2)', a1, a2, x)");
   w.factory("Exponential::bmodel(z, 1)");

   // signal model   
   w.factory("nsignal[100, 0.0, 1000.0]");
   //w.factory("mass[%f, %f, %f]' % (massguess, massmin, massmax))
   w.factory("mass[130, 110, 150]");
   w.factory("width[1, 0.5, 5]");
   w.factory("Gaussian::smodel(x, mass, width)");
   RooAbsPdf * smodel = w.pdf("smodel");

   w.factory("SUM::model(nbackground*bmodel, nsignal*smodel)");
   RooAbsPdf * model = w.pdf("model");
   RooRealVar * x = w.var("x");

In [ ]:
w.Print();

We create now the RooFit data set importing the data from the ROOT tree.
We import also the data set in the workspace

In [ ]:
RooDataSet data("data","data",*w.var("x"),RooFit::Import(tree) );
w.import(data);

we now fit the data

In [ ]:
RooFitResult * r = nullptr;
RooPlot * plot = nullptr;

In [ ]:
plot = w.var("x")->frame();
data.plotOn(plot);

In [ ]:
plot->Draw(); c1->Draw();

In [ ]:
r = model->fitTo(data, RooFit::Minimizer("Minuit2"),RooFit::Save(true), RooFit::Offset(true));

Plot of the fit result

In [ ]:

model->plotOn(plot);
model->plotOn(plot, RooFit::Components("bmodel"),RooFit::LineStyle(kDashed));
model->plotOn(plot, RooFit::Components("smodel"),RooFit::LineColor(kRed));
model->paramOn(plot);

plot->Draw();
c1->Draw();

Before saving the workspace we create the RooStats model config object which is needed for the RooStats tools.

In [ ]:
RooStats::ModelConfig mc("ModelConfig",&w);
mc.SetPdf(*w.pdf("model"));
mc.SetParametersOfInterest(*w.var("nsignal"));
mc.SetObservables(*w.var("x"));
// define set of nuisance parameters
w.defineSet("nuisParams","a1,a2,nbackground");

mc.SetNuisanceParameters(*w.set("nuisParams"));
mc.Print();

We want to semplify the H->gg model (for speeding-up ) the RooStats calculations, by assuming that the mass and the sigma of the signal are known and do not vary in the fits. We set them to constant

In [ ]:
w.var("mass")->setConstant(true);
w.var("width")->setConstant(true);
w.import(mc);

In [ ]:
w.writeToFile("HiggsModel.root",true);
w.Print();